# Semantic Segmentation 

This project detects road and not-road using a dataset and tensorflow. 

## Imports and gpu-check

In [1]:
#!/usr/bin/env python3
import time
import os.path
import tensorflow as tf
import helper
import warnings
from distutils.version import LooseVersion
import project_tests as tests
from datetime import timedelta
import scipy.misc

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer.  You are using {}'.format(tf.__version__)
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))


TensorFlow Version: 1.10.0
Default GPU Device: /device:GPU:0


## Parameters

Variations of ... did the following: ... 

In [2]:
L2_REG = 1e-5 # TODO: try smaller number like 1e-5
STDEV = 1e-3
KEEP_PROB = 0.5
LEARNING_RATE = 1e-4
EPOCHS = 20
BATCH_SIZE = 4 
IMAGE_SHAPE = (160, 576)
NUM_CLASSES = 2

DATA_DIR = './data'
RUNS_DIR = './runs'
MODEL_DIR = './models'

## load_vgg(sess, vgg_path)

In [3]:
def load_vgg(sess, vgg_path):
    """
    Load Pretrained VGG Model into TensorFlow.
    :param sess: TensorFlow Session
    :param vgg_path: Path to vgg folder, containing "variables/" and "saved_model.pb"
    :return: Tuple of Tensors from VGG model (image_input, keep_prob, layer3_out, layer4_out, layer7_out)
    """
    # TODO: Implement function
    #   Use tf.saved_model.loader.load to load the model and weights
    vgg_tag = 'vgg16'
    vgg_input_tensor_name = 'image_input:0'
    vgg_keep_prob_tensor_name = 'keep_prob:0'
    vgg_layer3_out_tensor_name = 'layer3_out:0'
    vgg_layer4_out_tensor_name = 'layer4_out:0'
    vgg_layer7_out_tensor_name = 'layer7_out:0'
    
    graph = tf.get_default_graph()

    tf.saved_model.loader.load(sess, [vgg_tag], vgg_path)
    image_input = graph.get_tensor_by_name(vgg_input_tensor_name)
    keep_prob = graph.get_tensor_by_name(vgg_keep_prob_tensor_name)
    layer3 = graph.get_tensor_by_name(vgg_layer3_out_tensor_name)
    layer4 = graph.get_tensor_by_name(vgg_layer4_out_tensor_name)
    layer7 = graph.get_tensor_by_name(vgg_layer7_out_tensor_name)
    return image_input, keep_prob, layer3, layer4, layer7

print("Load vgg16 model: ")
tests.test_load_vgg(load_vgg, tf)


Load vgg16 model: 
Tests Passed


In [4]:
def layers(vgg_layer3_out, vgg_layer4_out, vgg_layer7_out, num_classes): 
    """
    Create the layers for a fully convolutional network.  Build skip-layers using the vgg layers.
    :param vgg_layer3_out: TF Tensor for VGG Layer 3 output
    :param vgg_layer4_out: TF Tensor for VGG Layer 4 output
    :param vgg_layer7_out: TF Tensor for VGG Layer 7 output
    :param num_classes: Number of classes to classify
    :return: The Tensor for the last layer of output
    """
    # TODO: check good initializer and regularizer 
    kernel_regularizer = tf.contrib.layers.l2_regularizer(L2_REG)
    kernel_initializer = tf.initializers.orthogonal()
    
    # perform a convolution on layer 7 and connect it to the output
    layer7_conv_1x1 = tf.layers.conv2d(vgg_layer7_out, num_classes, 1,1, padding = 'same',
                                kernel_initializer= tf.random_normal_initializer(stddev=0.01), 
                                kernel_regularizer= tf.contrib.layers.l2_regularizer(1e-3))
    output = tf.layers.conv2d_transpose(layer7_conv_1x1, num_classes, 4, 2, padding ='same', 
                                kernel_initializer= tf.random_normal_initializer(stddev=0.01), 
                                kernel_regularizer= tf.contrib.layers.l2_regularizer(1e-3))
    
    # perform a convolution on layer 4 and add it to the existing output, then do a transposed conv
    layer4_conv_1x1 = tf.layers.conv2d(vgg_layer4_out, num_classes, 1,1, padding = 'same',
                                kernel_initializer= tf.random_normal_initializer(stddev=0.01), 
                                kernel_regularizer= tf.contrib.layers.l2_regularizer(1e-3))
    output = tf.add(output, layer4_conv_1x1)
    output = tf.layers.conv2d_transpose(output, num_classes, 4, 2, padding='same', 
                                kernel_initializer= tf.random_normal_initializer(stddev=0.01), 
                                kernel_regularizer= tf.contrib.layers.l2_regularizer(1e-3))
                                                                             
    # perform a convolution on layer 3 and add it to the existing output, then do a transposed conv
    layer3_conv_1x1 = tf.layers.conv2d(vgg_layer3_out, num_classes, 1,1, padding = 'same',
                                kernel_initializer= tf.random_normal_initializer(stddev=0.01), 
                                kernel_regularizer= tf.contrib.layers.l2_regularizer(1e-3))
    output = tf.add(output, layer3_conv_1x1)
    output = tf.layers.conv2d_transpose(output, num_classes, 16, 8, padding='same',                                       
                                kernel_initializer= tf.random_normal_initializer(stddev=0.01), 
                                kernel_regularizer= tf.contrib.layers.l2_regularizer(1e-3))
    
    #tf.Print(output, [tf.shape(output)[1:3]])
    return output
print("Load layers: ")
tests.test_layers(layers)

Load layers: 
Tests Passed


In [ ]:

def optimize(nn_last_layer, correct_label, learning_rate, num_classes):
    """
    Build the TensorFLow loss and optimizer operations.
    :param nn_last_layer: TF Tensor of the last layer in the neural network
    :param correct_label: TF Placeholder for the correct label image
    :param learning_rate: TF Placeholder for the learning rate
    :param num_classes: Number of classes to classify
    :return: Tuple of (logits, train_op, cross_entropy_loss)
    """
    logits = tf.reshape(nn_last_layer, (-1, num_classes))
    labels = tf.reshape(correct_label, (-1, num_classes))
    cross_entropy_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=correct_label))
    #  train_op = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9).minimize(cross_entropy_loss)
    train_op = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cross_entropy_loss)
    
    return logits, train_op, cross_entropy_loss
tests.test_optimize(optimize)


Tests Passed


In [ ]:
def train_nn(sess, epochs, batch_size, get_batches_fn, train_op, cross_entropy_loss, input_image,
             correct_label, keep_prob, learning_rate, saver, data_dir):
    """
    Train neural network and print out the loss during training.
    :param sess: TF Session
    :param epochs: Number of epochs
    :param batch_size: Batch size
    :param get_batches_fn: Function to get batches of training data.  Call using get_batches_fn(batch_size)
    :param train_op: TF Operation to train the neural network
    :param cross_entropy_loss: TF Tensor for the amount of loss
    :param input_image: TF Placeholder for input images
    :param correct_label: TF Placeholder for label images
    :param keep_prob: TF Placeholder for dropout keep probability
    :param learning_rate: TF Placeholder for learning rate
    """
    # TODO: Implement function
    for epoch in range(epochs):
        s_time = time.time()
        for image, targets in get_batches_fn(batch_size):
            _, loss = sess.run([train_op, cross_entropy_loss], 
                feed_dict = {input_image: image, correct_label: targets, keep_prob: KEEP_PROB ,
                             learning_rate: LEARNING_RATE })
        # Print data on the learning process
        print("Epoch: {}".format(epoch + 1), "/ {}".format(epochs), " Loss: {:.3f}".format(loss), " Time: ",
              str(timedelta(seconds=(time.time() - s_time))))
        if (epoch + 1) % 10 == 0: # Save every 5 epochs
            save_path = saver.save(sess, os.path.join(data_dir, 'epoch_' + str(epoch) + '.ckpt'))
        

#Don't use the provided test, as we have a different input to the function
#tests.test_train_nn(train_nn)

In [ ]:


def run():
    tests.test_for_kitti_dataset(DATA_DIR)
    # Download pretrained vgg model
    helper.maybe_download_pretrained_vgg(DATA_DIR)

    print("Training...")

    with tf.Session() as sess:
        # Path to vgg model
        vgg_path = os.path.join(DATA_DIR, 'vgg')
        # Create function to get batches
        get_batches_fn = helper.gen_batch_function(os.path.join(DATA_DIR, 'data_road/training'), IMAGE_SHAPE)
        # OPTIONAL: Augment Images for better results
        #  https://datascience.stackexchange.com/questions/5224/how-to-prepare-augment-images-for-neural-network
        # Add some augmentations, see helper.py
        input, keep_prob, layer3, layer4, layer7 = load_vgg(sess, vgg_path)
        output = layers(layer3, layer4, layer7, NUM_CLASSES)
        correct_label = tf.placeholder(dtype = tf.float32, shape = (None, None, None, NUM_CLASSES))
        learning_rate = tf.placeholder(dtype = tf.float32)
        logits, train_op, cross_entropy_loss = optimize(output, correct_label, learning_rate, NUM_CLASSES)
        tf.set_random_seed(123)
        sess.run(tf.global_variables_initializer())
        saver = tf.train.Saver() #Simple model saver
        train_nn(sess, EPOCHS, BATCH_SIZE, get_batches_fn, train_op, cross_entropy_loss, input, correct_label,
                 keep_prob, learning_rate,  saver, MODEL_DIR)
        # Save inference data using helper.save_inference_samples
        helper.save_inference_samples(RUNS_DIR, DATA_DIR, sess, IMAGE_SHAPE, logits, keep_prob, input)
        print("Done")
        # OPTIONAL: Apply the trained model to a video
        
run()

Tests Passed
Training...
INFO:tensorflow:Restoring parameters from ./data\vgg\variables\variables
Epoch: 1 / 20  Loss: 0.207  Time:  0:01:37.864654


### without regularization

Epoch: 1 / 5  Loss: 0.947  Time:  0:01:31.879395
Epoch: 2 / 5  Loss: 0.774  Time:  0:01:29.919057
Epoch: 3 / 5  Loss: 0.726  Time:  0:01:40.409835
Epoch: 4 / 5  Loss: 0.700  Time:  0:01:56.143895
Epoch: 5 / 5  Loss: 0.672  Time:  0:01:58.565175
Training Finished. Saving test images to: ./runs\1540840717.903083


### standard regularization

Epoch: 1 / 20  Loss: 0.135  Time:  0:01:42.530179
Epoch: 2 / 20  Loss: 0.255  Time:  0:02:06.080995
Epoch: 3 / 20  Loss: 0.089  Time:  0:02:09.374919
Epoch: 4 / 20  Loss: 0.073  Time:  0:02:09.156908
Epoch: 5 / 20  Loss: 0.075  Time:  0:02:09.304775
Epoch: 6 / 20  Loss: 0.083  Time:  0:02:10.274703
Epoch: 7 / 20  Loss: 0.135  Time:  0:02:09.564648
Epoch: 8 / 20  Loss: 0.036  Time:  0:02:09.255591
Epoch: 9 / 20  Loss: 0.044  Time:  0:02:09.056574
Epoch: 10 / 20  Loss: 0.090  Time:  0:02:09.841141
Epoch: 11 / 20  Loss: 0.049  Time:  0:02:08.956540
Epoch: 12 / 20  Loss: 0.048  Time:  0:02:09.494981
Epoch: 13 / 20  Loss: 0.067  Time:  0:02:09.884587
Epoch: 14 / 20  Loss: 0.015  Time:  0:02:09.849671
Epoch: 15 / 20  Loss: 0.026  Time:  0:02:10.587436
Epoch: 16 / 20  Loss: 0.089  Time:  0:02:09.789505
Epoch: 17 / 20  Loss: 0.038  Time:  0:02:09.382614
Epoch: 18 / 20  Loss: 0.049  Time:  0:02:09.362768
Epoch: 19 / 20  Loss: 0.063  Time:  0:02:09.163698
Epoch: 20 / 20  Loss: 0.052  Time:  0:02:09.201310
1540845167.5800643

### augment with flip and random brightness and contrast 


Epoch: 1 / 20  Loss: 0.746  Time:  0:01:31.271338
Epoch: 2 / 20  Loss: 0.586  Time:  0:01:32.809698
Epoch: 3 / 20  Loss: 0.457  Time:  0:01:52.036394
Epoch: 4 / 20  Loss: 0.320  Time:  0:02:04.046675
Epoch: 5 / 20  Loss: 0.165  Time:  0:02:09.064407
Epoch: 6 / 20  Loss: 0.129  Time:  0:02:11.825765
Epoch: 7 / 20  Loss: 0.080  Time:  0:02:08.195775
Epoch: 8 / 20  Loss: 0.102  Time:  0:02:08.505612
Epoch: 9 / 20  Loss: 0.117  Time:  0:02:08.363144
Epoch: 10 / 20  Loss: 0.073  Time:  0:02:08.413695
Epoch: 11 / 20  Loss: 0.059  Time:  0:02:04.330584
Epoch: 12 / 20  Loss: 0.099  Time:  0:02:08.333077
Epoch: 13 / 20  Loss: 0.078  Time:  0:02:09.456341
Epoch: 14 / 20  Loss: 0.086  Time:  0:02:09.155645
Epoch: 15 / 20  Loss: 0.072  Time:  0:02:18.026403
Epoch: 16 / 20  Loss: 0.053  Time:  0:02:09.169739
Epoch: 17 / 20  Loss: 0.091  Time:  0:02:18.399122
Epoch: 18 / 20  Loss: 0.078  Time:  0:02:14.184786
Epoch: 19 / 20  Loss: 0.136  Time:  0:02:17.844234
Epoch: 20 / 20  Loss: 0.191  Time:  0:02:13.351590
Training Finished. Saving test images to: ./runs\1540852572.6791086


looks like too little generalization or too little regularization


### regularization e-3 --> e-5 gets slightly better in iou
Epoch: 1 / 20  Loss: 0.790  Time:  0:01:33.200101
Epoch: 2 / 20  Loss: 0.691  Time:  0:01:23.696226
Epoch: 3 / 20  Loss: 0.571  Time:  0:01:32.057452
Epoch: 4 / 20  Loss: 0.394  Time:  0:01:38.021367
Epoch: 5 / 20  Loss: 0.449  Time:  0:01:46.768626
Epoch: 6 / 20  Loss: 0.259  Time:  0:01:54.783575
Epoch: 7 / 20  Loss: 0.113  Time:  0:01:55.703466
Epoch: 8 / 20  Loss: 0.089  Time:  0:01:59.253920
Epoch: 9 / 20  Loss: 0.162  Time:  0:02:04.317690
Epoch: 10 / 20  Loss: 0.128  Time:  0:02:07.141466
Epoch: 11 / 20  Loss: 0.110  Time:  0:01:54.742900
Epoch: 12 / 20  Loss: 0.222  Time:  0:02:16.932343
Epoch: 13 / 20  Loss: 0.105  Time:  0:02:09.088753
Epoch: 14 / 20  Loss: 0.156  Time:  0:02:08.976205
Epoch: 15 / 20  Loss: 0.043  Time:  0:02:09.306973
Epoch: 16 / 20  Loss: 0.132  Time:  0:02:09.489564
Epoch: 17 / 20  Loss: 0.050  Time:  0:02:09.450222
Epoch: 18 / 20  Loss: 0.100  Time:  0:02:09.112877
Epoch: 19 / 20  Loss: 0.124  Time:  0:02:09.056755
Epoch: 20 / 20  Loss: 0.159  Time:  0:02:09.210153
Training Finished. Saving test images to: ./runs\1540886071.6455922
Done

